# Import Libraries

In [1]:
import sys
sys.path.append('..')

In [ ]:
import random
from copy import deepcopy
from lib.solution import Solution
import pandas as pd

# Import Data

In [4]:
# Load artists data
artists_df = pd.read_csv(r"C:\Users\anama\OneDrive\Ambiente de Trabalho\Mestrado\S2\CIFO\data\artists(in).csv")

# Preview the first few rows
print(artists_df.head())

print(f"\n Number of artists: {len(artists_df)}")

   Unnamed: 0             name       genre  popularity
0           0    Midnight Echo        Rock          75
1           1      Solar Flare  Electronic          78
2           2     Velvet Pulse        Jazz          35
3           3     Neon Reverie  Electronic         100
4           4  The Silver Owls   Classical          85

 Number of artists: 35


In [5]:
# Rename the column
artists_df.rename(columns={"Unnamed: 0": "id"}, inplace=True)

# Confirm the change
print(artists_df.head())

   id             name       genre  popularity
0   0    Midnight Echo        Rock          75
1   1      Solar Flare  Electronic          78
2   2     Velvet Pulse        Jazz          35
3   3     Neon Reverie  Electronic         100
4   4  The Silver Owls   Classical          85


In [6]:
class Artist:
    def __init__(self, id, name, popularity, genre):
        self.id = id
        self.name = name
        self.popularity = popularity
        self.genre = genre

    def __repr__(self):
        return f"{self.name} (Pop: {self.popularity}, Genre: {self.genre})"

# Convert to list of Artist objects
artists = [
    Artist(row["id"], row["name"], row["popularity"], row["genre"])
    for _, row in artists_df.iterrows()
]

In [7]:
# Try to load with headers and index
conflict_df = pd.read_csv(r"C:\Users\anama\OneDrive\Ambiente de Trabalho\Mestrado\S2\CIFO\data\conflicts(in).csv", index_col=0)

# Convert to NumPy matrix for easier computation later
conflict_matrix = conflict_df.to_numpy()

# Check matrix size
print("Conflict matrix shape:", conflict_matrix.shape)

Conflict matrix shape: (35, 35)
